search jupyter notebooks using llama-gpt?

search llama?



In [ ]:
//import {fileURLToPath} from "url";
import path from "path";
import {getLlama, LlamaChatSession} from "node-llama-cpp";
import process from "process"
const importNotebook = importer.import
const storeLlamaFunction = importNotebook('store llama function')
const {getExports, selectAst, cacheCells} = importNotebook([
    'select code tree', 'get exports from source', 'cache notebook'])
const lookupCell = importer.lookupCell

const HOMEPATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
//const __dirname = path.dirname(fileURLToPath(import.meta.url));

let llama;
let model;

async function askLlamaAboutFunctions(query, functions, descriptions, session) {
    let q1 = "Given a list of functions:\n";
    for(let i = 0; i < functions.length; i++) {
        if(descriptions[i]) {
            q1 += functions[i] + ' - ' + descriptions[i] + '\n'
        } else {
            q1 += functions[i] + '\n'
        }
    }
    q1 += 'which one most closely matches the query \"' + query + '\"?'
    console.log("User: " + q1);
    const a1 = await session.prompt(q1);
    console.log("AI: " + a1);
    // TODO: parse function name
    let result = a1.split('"').filter(x => functions.indexOf(x) > -1)[0]
    return result
}



async function searchLlama(query) {
    if(!llama) {
        llama = await getLlama();
    }
    if(!model) {
        model = await llama.loadModel({
            modelPath: path.join(HOMEPATH, "llama.cpp", "models", "code-llama-7b-chat.gguf" /*"Meta-Llama-3.1-8B-Instruct.Q4_K_M.gguf"*/)
        });
    }

    const context = await model.createContext();
    const session = new LlamaChatSession({
        contextSequence: context.getSequence()
    });

    // TODO: use AI to binary/boolean merge search groups of functions for the query
    let cellCache = importNotebook('cell cache').cellCache
    let functions = []
    let descriptions = []
    for(let i = 0; i < cellCache.length; i++) {
        let cell = cellCache[i]
        if(!cell[2].questions || !cell[2].questions[0]) continue
        let code = lookupCell(cell[1], cacheCells)
        if(code.language != 'javascript') continue
        let rpcFunction
        try {
            rpcFunction = getExports(code.code)[0]
        } catch (e) {

        }
        if(!rpcFunction) continue

        functions[functions.length] = rpcFunction
        descriptions[descriptions.length] = code.questions.sort((a, b) => b.length - a.length)[0]
        
        if(functions.length == 20) {
            let result = await askLlamaAboutFunctions(query, functions, descriptions, session)
            descriptions = [descriptions[functions.indexOf(result)]]
            functions = [result]
        }
    }
    if(functions.length > 1) {
        let result = await askLlamaAboutFunctions(query, functions, descriptions, session)
        functions = [result]
    }
    
    console.log('Result: ', functions)
    // TODO: load multiple models based on the tool i'm using
    
    
    
    const q2 = "What did you just say?";
    console.log("User: " + q2);
    
    const a2 = await session.prompt(q2);
    console.log("AI: " + a2);
}

export default searchLlama



how to scan sqlite tools?





how to scan chat logs?





how to scan code history?







store llama function?



In [ ]:

function storeLlamaFunction () {
  
}

module.exports = {
  storeLlamaFunction
}



how to scan calendar journal entries?

how to ask if my theories are correct?


